In [2]:
import pandas as pd

df = pd.read_csv("../../data/eff_pop.csv", sep=";")
df_metadata = pd.read_csv("../../data/metadata_eff_pop.csv", sep=";")

In [3]:
df.head()

,GEO,GEO_OBJECT,SEX,AGE,EP_MEASURE,OBS_STATUS_FR,TIME_PERIOD,OBS_VALUE
0,78,DEP,M,Y50T54,POP,D,2007,44919.0
1,85,DEP,M,Y60T64,POP,D,2007,15680.0
2,82,DEP,F,Y15T19,POP,D,2007,6516.0
3,79,DEP,_T,Y25T29,POP,D,2007,19285.0
4,62,DEP,F,Y_LT5,POP,D,2007,47222.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272973 entries, 0 to 272972
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   GEO            272973 non-null  object 
 1   GEO_OBJECT     272973 non-null  object 
 2   SEX            272973 non-null  object 
 3   AGE            272973 non-null  object 
 4   EP_MEASURE     272973 non-null  object 
 5   OBS_STATUS_FR  272973 non-null  object 
 6   TIME_PERIOD    272973 non-null  int64  
 7   OBS_VALUE      272973 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 16.7+ MB


In [5]:
df_metadata.head()

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD
0,AGE,Âge,Y_LT20,Moins de 20 ans
1,AGE,Âge,Y20T39,De 20 à 39 ans
2,AGE,Âge,Y40T59,De 40 à 59 ans
3,AGE,Âge,Y60T74,De 60 à 74 ans
4,AGE,Âge,Y_GE75,75 ans ou plus


In [6]:
df["EP_MEASURE"].value_counts()

EP_MEASURE
POP                   272211
PT_YGE65_IN_Y20T64       552
AVERAGE                  105
MEDAGE                   105
Name: count, dtype: int64

In [7]:
df["GEO_OBJECT"].value_counts()

GEO_OBJECT
DEP       226221
REG        39657
FRANCE      4800
OTHER       2295
Name: count, dtype: int64

In [8]:
df_metadata["COD_MOD"].value_counts()

COD_MOD
_T          2
F           2
93          2
94          2
84          2
           ..
973         1
974         1
DOM         1
976         1
F_X_D976    1
Name: count, Length: 175, dtype: int64

Des doublons imprévus...

Il est plus prudent de séparer ce fichier en plusieurs tables metadata. 

In [11]:
df= df.rename(columns={
    "GEO": "Num_dep",
    "SEX" : "Sexe",
    "GEO_OBJECT": "Carac_dep",
    "AGE": "Age",
    "EP_MEASURE": "Carac_mesure",
    "OBS_STATUS_FR": "Chiffre_def",
    "TIME_PERIOD": "Annee",
    "OBS_VALUE": "Effectif" })

In [12]:
for col in df.select_dtypes(include="object").columns:
    max_len = df[col].astype(str).str.len().max()
    print(f"{col} : VARCHAR({max_len})")


Num_dep : VARCHAR(8)
Carac_dep : VARCHAR(6)
Sexe : VARCHAR(2)
Age : VARCHAR(6)
Carac_mesure : VARCHAR(18)
Chiffre_def : VARCHAR(4)


In [13]:
for col in df.select_dtypes(include="int").columns:
    max_val = df[col].max()
    min_val = df[col].min()
    max_digits = max(len(str(abs(max_val))), len(str(abs(min_val))))
    print(f"{col} : INTEGER (min={min_val}, max={max_val}, digits={max_digits})")


Annee : INTEGER (min=1990, max=2025, digits=4)


In [14]:
for col in df.select_dtypes(include="float").columns:
    max_val = df[col].max()
    min_val = df[col].min()

    def count_digits(val):
        if pd.isna(val):
            return (0, 0)
        parts = f"{val:.10f}".rstrip("0").split(".")
        return (len(parts[0]), len(parts[1]) if len(parts) > 1 else 0)

    entiers, decimales = zip(*df[col].dropna().map(count_digits))

    print(f"{col} : NUMERIC({max(entiers)+max(decimales)}, {max(decimales)}) "
          f"(min={min_val}, max={max_val})")


Effectif : NUMERIC(14, 6) (min=0.060784, max=68605616.0)
